In [1]:
import pandas as pd
import numpy as np

In [2]:
#u.date에 있는 정보 확인
cols = ['user-id','MovieId','Rating','TimeStamp']
ds = pd.read_csv('ml-100k/u.data',sep = '\t',header = None, names=cols)
ds

,user-id,MovieId,Rating,TimeStamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [3]:
#u.user에 있는 정보 확인
userInfo=['user-id', 'age', 'gender','occupation','zip-code']
userInformation=pd.read_csv('ml-100k/u.user', sep='\|', header=None, engine='python', names=userInfo)
userInformation

,user-id,age,gender,occupation,zip-code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [4]:
# 어떤 유저가 어느 영화에 평가했는지 보기 위해 유저 데이터 합침 (영화제목은 아직 모르는 상태)
allUserInfo=pd.merge(ds, userInformation, on='user-id')
allUserInfo

,user-id,MovieId,Rating,TimeStamp,age,gender,occupation,zip-code
0,196,242,3,881250949,49,M,writer,55105
1,196,393,4,881251863,49,M,writer,55105
2,196,381,4,881251728,49,M,writer,55105
3,196,251,3,881251274,49,M,writer,55105
4,196,655,5,881251793,49,M,writer,55105
...,...,...,...,...,...,...,...,...
99995,941,919,5,875048887,20,M,student,97229
99996,941,273,3,875049038,20,M,student,97229
99997,941,1,5,875049144,20,M,student,97229
99998,941,294,4,875048532,20,M,student,97229


In [5]:
#u.item에서 영화id와 영화 제목만 가져옴
name = ['MovieId','Names']
movies = pd.read_csv('ml-100k/u.item',sep = '\|',header = None,engine = 'python',usecols=[0,1],names = name)
movies

,MovieId,Names
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


In [6]:
# Make a list of the top 100 most popular movies
# Extract a list of movies with the highest number of ratings 1) for all users

# 우선 영화의 이름도 출력하기 위해 직전에 얻어낸 영화제목 정보와 유저데이터를 합칩니다
userInfoAndMovie=pd.merge(allUserInfo, movies, on='MovieId')

#각각의 영화들에 대해 유져들이 평가해준 횟수를 추출하고, 새로운 열을 만들어 각각의 영화에 평가 횟수를 저장해줍니다
userInfoAndMovie['Rating count']=userInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

#중복되는 영화 제목을 제거해주면, 모든 유저가 평가한 영화중에 평가 횟수가 가장 많은 상위 100개의 영화를 출력해줍니다.
userInfoAndMovie[['MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,MovieId,Names,Rating count
33307,50,Star Wars (1977),583
17679,258,Contact (1997),509
11938,100,Fargo (1996),508
30133,181,Return of the Jedi (1983),507
18937,294,Liar Liar (1997),485
...,...,...,...
85463,678,Volcano (1997),219
13731,95,Aladdin (1992),219
2850,8,Babe (1995),219
62916,427,To Kill a Mockingbird (1962),219


In [7]:
# Extract a list of movies with the highest number of ratings 2) by gender

#우선 성별이 남자인 유저들을 그 전에 저장해 두었던 user정보에서 뽑아내줍니다
menInfo=allUserInfo.loc[allUserInfo['gender']=='M']

#영화의 이름도 출력하기 위해 직전에 얻어낸 영화제목 정보와 남자유저데이터를 합칩니다
menInfoAndMovie=pd.merge(menInfo, movies, on='MovieId')

#각각의 영화들에 대해 남자 유져들이 평가해준 횟수를 추출하고, 새로운 열을 만들어 각각의 영화에 평가 횟수를 저장해줍니다
menInfoAndMovie['Rating count']=menInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

#중복되는 영화 제목을 제거해주면, 모든 남자 유저가 평가한 영화중에 평가 횟수가 가장 많은 상위 100개의 영화를 출력해줍니다.
menInfoAndMovie[['gender','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,gender,MovieId,Names,Rating count
18359,M,50,Star Wars (1977),432
15295,M,181,Return of the Jedi (1983),383
21462,M,100,Fargo (1996),383
5083,M,258,Contact (1997),372
6824,M,294,Liar Liar (1997),344
...,...,...,...,...
37236,M,323,Dante's Peak (1997),167
24053,M,508,"People vs. Larry Flynt, The (1996)",165
332,M,655,Stand by Me (1986),165
63084,M,678,Volcano (1997),165


In [8]:
#성별이 여자인 유저들을 그 전에 저장해 두었던 user정보에서 뽑아내줍니다
menInfo=allUserInfo.loc[allUserInfo['gender']=='F']

#직전과 동일하게 수행합니다
menInfoAndMovie=pd.merge(menInfo, movies, on='MovieId')
menInfoAndMovie['Rating count']=menInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

#중복되는 영화 제목을 제거해주면, 모든 여자 유저가 평가한 영화중에 평가 횟수가 가장 많은 상위 100개의 영화를 출력해줍니다.
menInfoAndMovie[['gender','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,gender,MovieId,Names,Rating count
8816,F,286,"English Patient, The (1996)",152
4130,F,50,Star Wars (1977),151
1174,F,288,Scream (1996),143
3359,F,294,Liar Liar (1997),141
2979,F,258,Contact (1997),137
...,...,...,...,...
14520,F,321,Mother (1996),57
4174,F,183,Alien (1979),56
8525,F,176,Aliens (1986),55
19313,F,690,Seven Years in Tibet (1997),55


In [9]:
# Extract a list of movies with the highest number of ratings 3) by occupation.
#우선 직업이 어떤 종류가 있나 확인해주었습니다, 편의를 위해 리스트로 만들어 줍니다
occupation = pd.read_csv('ml-100k/u.occupation', sep="|", header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [10]:
# 만든 리스트를 편리를 위해 각각의 직업명을 리스트에서 받아와
# for문으로 처리하면 동작할줄 알았는데 동작하지 않습니다, 함수로 만들어도 동작하지 않았습니다
for i in occupation_list:
    
    jobInfo=allUserInfo[allUserInfo['occupation']==i]
    jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
    
    jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')
    jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

In [11]:
#어쩔수 없이 occupation의 종류만 바꿔가면서 처리했습니다.

#administrator
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='administrator']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
1182,administrator,286,"English Patient, The (1996)",47
1757,administrator,50,Star Wars (1977),44
1253,administrator,181,Return of the Jedi (1983),40
2839,administrator,237,Jerry Maguire (1996),38
314,administrator,258,Contact (1997),38
...,...,...,...,...
2549,administrator,474,Dr. Strangelove or: How I Learned to Stop Worr...,18
3970,administrator,259,George of the Jungle (1997),18
1969,administrator,83,Much Ado About Nothing (1993),18
1102,administrator,125,Phenomenon (1996),18


In [12]:
#artist
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='artist']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
66,artist,50,Star Wars (1977),18
735,artist,268,Chasing Amy (1997),17
704,artist,294,Liar Liar (1997),16
1012,artist,181,Return of the Jedi (1983),16
493,artist,258,Contact (1997),15
...,...,...,...,...
91,artist,435,Butch Cassidy and the Sundance Kid (1969),6
698,artist,275,Sense and Sensibility (1995),6
577,artist,462,Like Water For Chocolate (Como agua para choco...,6
604,artist,705,Singin' in the Rain (1952),6


In [13]:
#doctor
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='doctor']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
330,doctor,313,Titanic (1997),5
55,doctor,286,"English Patient, The (1996)",5
206,doctor,300,Air Force One (1997),5
68,doctor,742,Ransom (1996),4
72,doctor,297,Ulee's Gold (1997),4
...,...,...,...,...
47,doctor,270,Gattaca (1997),2
45,doctor,26,"Brothers McMullen, The (1995)",2
30,doctor,921,Farewell My Concubine (1993),1
447,doctor,595,"Fan, The (1996)",1


In [14]:
#educator
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='educator']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
299,educator,286,"English Patient, The (1996)",61
2902,educator,100,Fargo (1996),54
3872,educator,50,Star Wars (1977),50
111,educator,258,Contact (1997),45
2727,educator,127,"Godfather, The (1972)",43
...,...,...,...,...
401,educator,313,Titanic (1997),22
2561,educator,13,Mighty Aphrodite (1995),22
5368,educator,523,Cool Hand Luke (1967),22
3923,educator,209,This Is Spinal Tap (1984),22


In [15]:
#engineer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='engineer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
710,engineer,50,Star Wars (1977),50
3335,engineer,1,Toy Story (1995),46
1583,engineer,100,Fargo (1996),45
1126,engineer,174,Raiders of the Lost Ark (1981),44
600,engineer,181,Return of the Jedi (1983),44
...,...,...,...,...
3244,engineer,655,Stand by Me (1986),20
5830,engineer,215,Field of Dreams (1989),20
1796,engineer,527,Gandhi (1982),20
4021,engineer,742,Ransom (1996),20


In [16]:
#entertainment
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='entertainment']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
951,entertainment,268,Chasing Amy (1997),13
930,entertainment,258,Contact (1997),13
459,entertainment,288,Scream (1996),13
1378,entertainment,302,L.A. Confidential (1997),12
661,entertainment,294,Liar Liar (1997),11
...,...,...,...,...
1418,entertainment,270,Gattaca (1997),5
95,entertainment,289,Evita (1996),5
93,entertainment,219,"Nightmare on Elm Street, A (1984)",5
1528,entertainment,447,Carrie (1976),5


In [17]:
#executive
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='executive']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
489,executive,258,Contact (1997),22
565,executive,294,Liar Liar (1997),21
135,executive,237,Jerry Maguire (1996),19
781,executive,286,"English Patient, The (1996)",18
80,executive,322,Murder at 1600 (1997),18
...,...,...,...,...
1571,executive,521,"Deer Hunter, The (1978)",8
1530,executive,458,Nixon (1995),8
1688,executive,64,"Shawshank Redemption, The (1994)",8
456,executive,291,Absolute Power (1997),8


In [18]:
#healthcare
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='healthcare']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
589,healthcare,50,Star Wars (1977),11
363,healthcare,181,Return of the Jedi (1983),11
917,healthcare,300,Air Force One (1997),11
848,healthcare,288,Scream (1996),10
1232,healthcare,127,"Godfather, The (1972)",10
...,...,...,...,...
1389,healthcare,527,Gandhi (1982),5
1439,healthcare,318,Schindler's List (1993),5
301,healthcare,234,Jaws (1975),5
1406,healthcare,520,"Great Escape, The (1963)",5


In [19]:
#lawyer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='lawyer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
517,lawyer,286,"English Patient, The (1996)",9
197,lawyer,269,"Full Monty, The (1997)",9
86,lawyer,294,Liar Liar (1997),8
771,lawyer,258,Contact (1997),8
310,lawyer,50,Star Wars (1977),8
...,...,...,...,...
150,lawyer,99,Snow White and the Seven Dwarfs (1937),3
577,lawyer,265,"Hunt for Red October, The (1990)",3
1015,lawyer,89,Blade Runner (1982),3
122,lawyer,509,My Left Foot (1989),3


In [20]:
#librarian
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='librarian']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
1183,librarian,286,"English Patient, The (1996)",35
57,librarian,50,Star Wars (1977),28
1257,librarian,313,Titanic (1997),27
1108,librarian,269,"Full Monty, The (1997)",26
677,librarian,100,Fargo (1996),26
...,...,...,...,...
701,librarian,15,Mr. Holland's Opus (1995),12
3318,librarian,528,"Killing Fields, The (1984)",12
1797,librarian,792,Bullets Over Broadway (1994),12
3180,librarian,194,"Sting, The (1973)",12


In [21]:
#marketing
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='marketing']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
202,marketing,258,Contact (1997),18
274,marketing,286,"English Patient, The (1996)",17
139,marketing,300,Air Force One (1997),16
418,marketing,294,Liar Liar (1997),15
156,marketing,328,Conspiracy Theory (1997),14
...,...,...,...,...
493,marketing,228,Star Trek: The Wrath of Khan (1982),5
1786,marketing,243,Jungle2Jungle (1997),5
1507,marketing,343,Alien: Resurrection (1997),5
114,marketing,13,Mighty Aphrodite (1995),5


In [22]:
#none
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='none']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
226,none,258,Contact (1997),7
240,none,117,"Rock, The (1996)",6
305,none,300,Air Force One (1997),6
119,none,222,Star Trek: First Contact (1996),6
50,none,294,Liar Liar (1997),6
...,...,...,...,...
150,none,64,"Shawshank Redemption, The (1994)",3
483,none,174,Raiders of the Lost Ark (1981),2
502,none,270,Gattaca (1997),2
487,none,443,"Birds, The (1963)",2


In [23]:
#other
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='other']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
5105,other,50,Star Wars (1977),62
4434,other,300,Air Force One (1997),58
587,other,288,Scream (1996),56
3535,other,286,"English Patient, The (1996)",55
2468,other,294,Liar Liar (1997),55
...,...,...,...,...
320,other,235,Mars Attacks! (1996),23
5168,other,319,Everyone Says I Love You (1996),23
80,other,603,Rear Window (1954),23
7855,other,515,"Boot, Das (1981)",23


In [24]:
#programmer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='programmer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
214,programmer,50,Star Wars (1977),50
1001,programmer,100,Fargo (1996),47
2061,programmer,181,Return of the Jedi (1983),44
1021,programmer,174,Raiders of the Lost Ark (1981),42
3023,programmer,222,Star Trek: First Contact (1996),37
...,...,...,...,...
2733,programmer,197,"Graduate, The (1967)",18
5760,programmer,273,Heat (1995),18
287,programmer,187,"Godfather: Part II, The (1974)",18
4075,programmer,226,Die Hard 2 (1990),18


In [25]:
#retired
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='retired']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
1007,retired,100,Fargo (1996),10
205,retired,294,Liar Liar (1997),9
246,retired,237,Jerry Maguire (1996),8
77,retired,50,Star Wars (1977),8
1162,retired,127,"Godfather, The (1972)",8
...,...,...,...,...
325,retired,265,"Hunt for Red October, The (1990)",4
1062,retired,629,Victor/Victoria (1982),4
312,retired,179,"Clockwork Orange, A (1971)",4
326,retired,77,"Firm, The (1993)",4


In [26]:
#salesman
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='salesman']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
24,salesman,294,Liar Liar (1997),8
217,salesman,286,"English Patient, The (1996)",6
1,salesman,327,Cop Land (1997),6
133,salesman,313,Titanic (1997),6
93,salesman,748,"Saint, The (1997)",5
...,...,...,...,...
499,salesman,85,"Ref, The (1994)",2
481,salesman,871,Vegas Vacation (1997),2
479,salesman,368,Bio-Dome (1996),2
477,salesman,585,Son in Law (1993),2


In [27]:
#scientist
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='scientist']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
528,scientist,50,Star Wars (1977),19
449,scientist,100,Fargo (1996),18
587,scientist,286,"English Patient, The (1996)",17
1015,scientist,258,Contact (1997),17
170,scientist,181,Return of the Jedi (1983),15
...,...,...,...,...
493,scientist,4,Get Shorty (1995),6
746,scientist,187,"Godfather: Part II, The (1974)",6
798,scientist,13,Mighty Aphrodite (1995),6
706,scientist,183,Alien (1979),6


In [28]:
#student
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='student']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
299,student,288,Scream (1996),137
2647,student,50,Star Wars (1977),132
5031,student,181,Return of the Jedi (1983),121
878,student,258,Contact (1997),117
11337,student,294,Liar Liar (1997),117
...,...,...,...,...
1293,student,655,Stand by Me (1986),48
5859,student,179,"Clockwork Orange, A (1971)",48
12728,student,185,Psycho (1960),48
6936,student,301,In & Out (1997),47


In [29]:
#technician
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='technician']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
1036,technician,50,Star Wars (1977),22
538,technician,181,Return of the Jedi (1983),19
1453,technician,121,Independence Day (ID4) (1996),18
210,technician,100,Fargo (1996),18
964,technician,174,Raiders of the Lost Ark (1981),17
...,...,...,...,...
415,technician,180,Apocalypse Now (1979),9
789,technician,471,Courage Under Fire (1996),9
2959,technician,568,Speed (1994),9
2347,technician,182,GoodFellas (1990),9


In [30]:
#writer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='writer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating count']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('count')

jobInfoAndMovie[['occupation','MovieId','Names','Rating count']].sort_values(by='Rating count', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating count
1256,writer,50,Star Wars (1977),27
1802,writer,288,Scream (1996),27
1913,writer,100,Fargo (1996),27
1049,writer,181,Return of the Jedi (1983),25
481,writer,294,Liar Liar (1997),25
...,...,...,...,...
746,writer,195,"Terminator, The (1984)",12
536,writer,403,Batman (1989),12
1426,writer,191,Amadeus (1984),12
2317,writer,447,Carrie (1976),11


In [31]:
#Extract a list of movies with the highest average ratings 1) for all users

# 앞선 방법과 동일하게 진행하고 단지 transform 함수에 속성을 mean으로 바꾸어주었습니다
userInfoAndMovie=pd.merge(allUserInfo, movies, on='MovieId')

userInfoAndMovie['Rating Average']=userInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

#중복되는 영화 제목을 제거해주면, 모든 유저가 평가한 영화중에 평점 평균이 높은 상위 100개의 영화를 출력해줍니다.
userInfoAndMovie[['MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,MovieId,Names,Rating Average
99547,1293,Star Kid (1997),5.000000
98868,1122,They Made Me a Criminal (1939),5.000000
99964,1653,Entertaining Angels: The Dorothy Day Story (1996),5.000000
99205,1201,Marlene Dietrich: Shadow and Light (1996),5.000000
98438,1189,Prefontaine (1997),5.000000
...,...,...,...
69026,659,Arsenic and Old Lace (1944),4.078261
62384,132,"Wizard of Oz, The (1939)",4.077236
26299,651,Glory (1989),4.076023
25803,168,Monty Python and the Holy Grail (1974),4.066456


In [32]:
#Extract a list of movies with the highest average ratings 2) by gender

#우선 성별이 남자인 유저들을 그 전에 저장해 두었던 user정보에서 뽑아내줍니다
menInfo=allUserInfo.loc[allUserInfo['gender']=='M']

menInfoAndMovie=pd.merge(menInfo, movies, on='MovieId')

menInfoAndMovie['Rating Average']=menInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

#중복되는 영화 제목을 제거해주면, 모든 남자 유저가 평가한 영화중에 평점평균이 높은 상위 100개의 영화를 출력해줍니다.
menInfoAndMovie[['gender','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,gender,MovieId,Names,Rating Average
73685,M,1500,Santa with Muscles (1996),5.000000
73808,M,1201,Marlene Dietrich: Shadow and Light (1996),5.000000
73399,M,814,"Great Day in Harlem, A (1994)",5.000000
73246,M,1189,Prefontaine (1997),5.000000
73573,M,1122,They Made Me a Criminal (1939),5.000000
...,...,...,...,...
52361,M,166,Manon of the Spring (Manon des sources) (1986),4.146341
73177,M,1167,"Sum of Us, The (1994)",4.142857
54297,M,520,"Great Escape, The (1963)",4.138614
69286,M,656,M (1931),4.135135


In [33]:
#성별이 여자인 유저들을 그 전에 저장해 두었던 user정보에서 뽑아내줍니다
womanInfo=allUserInfo.loc[allUserInfo['gender']=='F']

womanInfoAndMovie=pd.merge(womanInfo, movies, on='MovieId')

womanInfoAndMovie['Rating Average']=womanInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

#중복되는 영화 제목을 제거해주면, 모든 여자 유저가 평가한 영화중에 평점평균이 높은 상위 100개의 영화를 출력해줍니다.
womanInfoAndMovie[['gender','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,gender,MovieId,Names,Rating Average
24871,F,1368,Mina Tannenbaum (1994),5.0
24994,F,1451,Foreign Correspondent (1940),5.0
25419,F,1594,Everest (1998),5.0
25542,F,1472,"Visitors, The (Visiteurs, Les) (1993)",5.0
25465,F,1599,Someone Else's America (1995),5.0
...,...,...,...,...
20126,F,317,In the Name of the Father (1993),4.1
23807,F,936,Brassed Off (1996),4.1
16962,F,515,"Boot, Das (1981)",4.1
21449,F,516,Local Hero (1983),4.1


In [34]:
#Extract a list of movies with the highest average ratings 3) by occupation.
#artist
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='artist']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
482,artist,891,Bent (1997),5.000000
1337,artist,242,Kolya (1996),5.000000
1132,artist,1404,Withnail and I (1987),5.000000
1644,artist,921,Farewell My Concubine (1993),5.000000
1978,artist,1084,Anne Frank Remembered (1995),5.000000
...,...,...,...,...
399,artist,423,E.T. the Extra-Terrestrial (1982),4.571429
1451,artist,269,"Full Monty, The (1997)",4.571429
646,artist,657,"Manchurian Candidate, The (1962)",4.571429
44,artist,484,"Maltese Falcon, The (1941)",4.571429


In [35]:
#administrator
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='administrator']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
7303,administrator,1449,Pather Panchali (1955),5.000000
7447,administrator,1127,"Truman Show, The (1998)",5.000000
7173,administrator,899,"Winter Guest, The (1997)",5.000000
7170,administrator,1024,Mrs. Dalloway (1997),5.000000
6679,administrator,224,Ridicule (1996),5.000000
...,...,...,...,...
4786,administrator,496,It's a Wonderful Life (1946),4.470588
704,administrator,100,Fargo (1996),4.444444
1475,administrator,55,"Professional, The (1994)",4.444444
2011,administrator,98,"Silence of the Lambs, The (1991)",4.441176


In [36]:
#doctor
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='doctor']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
467,doctor,989,Cats Don't Dance (1997),5.0
462,doctor,680,Kull the Conqueror (1997),5.0
398,doctor,132,"Wizard of Oz, The (1939)",5.0
155,doctor,433,Heathers (1989),5.0
153,doctor,173,"Princess Bride, The (1987)",5.0
...,...,...,...,...
207,doctor,1506,Nelly & Monsieur Arnaud (1995),4.0
208,doctor,484,"Maltese Falcon, The (1941)",4.0
409,doctor,427,To Kill a Mockingbird (1962),4.0
335,doctor,97,Dances with Wolves (1990),4.0


In [37]:
#educator
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='educator']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
9441,educator,119,Maya Lin: A Strong Clear Vision (1994),5.000000
8929,educator,1405,Boy's Life 2 (1997),5.000000
8637,educator,1201,Marlene Dietrich: Shadow and Light (1996),5.000000
8229,educator,793,Crooklyn (1994),5.000000
1974,educator,361,Incognito (1997),5.000000
...,...,...,...,...
8553,educator,607,Rebecca (1940),4.363636
6113,educator,60,Three Colors: Blue (1993),4.363636
4179,educator,64,"Shawshank Redemption, The (1994)",4.344828
5915,educator,480,North by Northwest (1959),4.333333


In [38]:
#engineer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='engineer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
8174,engineer,1203,Top Hat (1935),5.000000
8075,engineer,611,Laura (1944),5.000000
7327,engineer,1144,"Quiet Room, The (1996)",5.000000
7305,engineer,1143,Hard Eight (1996),5.000000
7839,engineer,1410,Harlem (1993),5.000000
...,...,...,...,...
7352,engineer,303,Ulee's Gold (1997),4.142857
40,engineer,8,Babe (1995),4.142857
1027,engineer,56,Pulp Fiction (1994),4.138889
507,engineer,187,"Godfather: Part II, The (1974)",4.125000


In [39]:
#entertainment
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='entertainment']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
2017,entertainment,483,Casablanca (1942),5.0
2008,entertainment,1062,Four Days in September (1997),5.0
1641,entertainment,657,"Manchurian Candidate, The (1962)",5.0
1691,entertainment,946,"Fox and the Hound, The (1981)",5.0
1662,entertainment,602,"American in Paris, An (1951)",5.0
...,...,...,...,...
1686,entertainment,498,"African Queen, The (1951)",4.5
1706,entertainment,480,North by Northwest (1959),4.5
1280,entertainment,13,Mighty Aphrodite (1995),4.5
1273,entertainment,433,Heathers (1989),4.5


In [40]:
#executive
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='executive']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
3402,executive,1643,Angel Baby (1995),5.000000
1875,executive,490,To Catch a Thief (1955),5.000000
3286,executive,746,Real Genius (1985),5.000000
3252,executive,863,"Garden of Finzi-Contini, The (Giardino dei Fin...",5.000000
2868,executive,543,"Mis�rables, Les (1995)",5.000000
...,...,...,...,...
3208,executive,650,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",4.333333
1541,executive,488,Sunset Blvd. (1950),4.333333
1819,executive,170,Cinema Paradiso (1988),4.333333
1621,executive,520,"Great Escape, The (1963)",4.333333


In [41]:
#healthcare
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='healthcare']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
2639,healthcare,320,Paradise Lost: The Child Murders at Robin Hood...,5.0
2550,healthcare,898,"Postman, The (1997)",5.0
1803,healthcare,452,Jaws 2 (1978),5.0
701,healthcare,615,"39 Steps, The (1935)",5.0
2654,healthcare,1368,Mina Tannenbaum (1994),5.0
...,...,...,...,...
1597,healthcare,692,"American President, The (1995)",4.0
1505,healthcare,630,"Great Race, The (1965)",4.0
1503,healthcare,651,Glory (1989),4.0
711,healthcare,617,"Blue Angel, The (Blaue Engel, Der) (1930)",4.0


In [42]:
#homemaker
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='homemaker']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
298,homemaker,309,Deceiver (1997),5.0
280,homemaker,762,Beautiful Girls (1996),5.0
251,homemaker,845,That Thing You Do! (1996),5.0
249,homemaker,319,Everyone Says I Love You (1996),5.0
240,homemaker,1152,In Love and War (1996),5.0
...,...,...,...,...
94,homemaker,261,Air Bud (1997),3.0
83,homemaker,242,Kolya (1996),3.0
267,homemaker,981,Dangerous Ground (1997),3.0
270,homemaker,676,"Crucible, The (1996)",3.0


In [43]:
#lawyer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='lawyer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
252,lawyer,603,Rear Window (1954),5.0
1218,lawyer,837,Meet John Doe (1941),5.0
513,lawyer,12,"Usual Suspects, The (1995)",5.0
523,lawyer,179,"Clockwork Orange, A (1971)",5.0
936,lawyer,856,Night on Earth (1991),5.0
...,...,...,...,...
1023,lawyer,91,"Nightmare Before Christmas, The (1993)",4.5
253,lawyer,127,"Godfather, The (1972)",4.5
475,lawyer,470,Tombstone (1993),4.5
459,lawyer,654,Chinatown (1974),4.5


In [44]:
#librarian
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='librarian']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
3196,librarian,998,Cabin Boy (1994),5.000000
4534,librarian,1217,Assassins (1995),5.000000
5205,librarian,656,M (1931),5.000000
4430,librarian,924,White Squall (1996),5.000000
5085,librarian,1449,Pather Panchali (1955),5.000000
...,...,...,...,...
4419,librarian,676,"Crucible, The (1996)",4.285714
4238,librarian,223,Sling Blade (1996),4.285714
4737,librarian,61,Three Colors: White (1994),4.285714
1291,librarian,923,Raise the Red Lantern (1991),4.285714


In [45]:
#marketing
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='marketing']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
556,marketing,378,Miracle on 34th Street (1994),5.000000
1601,marketing,1358,The Deadly Cure (1996),5.000000
699,marketing,942,What's Love Got to Do with It (1993),5.000000
700,marketing,838,In the Line of Duty 2 (1987),5.000000
1685,marketing,525,"Big Sleep, The (1946)",5.000000
...,...,...,...,...
675,marketing,196,Dead Poets Society (1989),4.166667
935,marketing,69,Forrest Gump (1994),4.142857
833,marketing,483,Casablanca (1942),4.111111
263,marketing,302,L.A. Confidential (1997),4.090909


In [46]:
#none
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='none']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
517,none,627,Robin Hood: Prince of Thieves (1991),5.000000
519,none,143,"Sound of Music, The (1965)",5.000000
152,none,64,"Shawshank Redemption, The (1994)",5.000000
871,none,603,Rear Window (1954),5.000000
872,none,178,12 Angry Men (1957),5.000000
...,...,...,...,...
567,none,269,"Full Monty, The (1997)",4.666667
719,none,96,Terminator 2: Judgment Day (1991),4.666667
204,none,496,It's a Wonderful Life (1946),4.500000
824,none,591,Primal Fear (1996),4.500000


In [47]:
#other
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='other']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
10432,other,903,Afterglow (1997),5.000000
10417,other,1617,Hugo Pool (1997),5.000000
10433,other,1295,Kicked in the Head (1997),5.000000
10220,other,1121,"Umbrellas of Cherbourg, The (Parapluies de Che...",5.000000
10233,other,1599,Someone Else's America (1995),5.000000
...,...,...,...,...
554,other,83,Much Ado About Nothing (1993),4.333333
8140,other,45,Eat Drink Man Woman (1994),4.333333
9426,other,653,Touch of Evil (1958),4.333333
9634,other,1007,Waiting for Guffman (1996),4.333333


In [48]:
#programmer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='programmer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
7644,programmer,975,Fear (1996),5.000000
6947,programmer,114,Wallace & Gromit: The Best of Aardman Animatio...,5.000000
2004,programmer,793,Crooklyn (1994),5.000000
7473,programmer,1114,Faithful (1996),5.000000
5390,programmer,1137,Beautiful Thing (1996),5.000000
...,...,...,...,...
3220,programmer,365,Powder (1995),4.250000
2808,programmer,474,Dr. Strangelove or: How I Learned to Stop Worr...,4.238095
2571,programmer,709,Strictly Ballroom (1992),4.230769
4394,programmer,132,"Wizard of Oz, The (1939)",4.227273


In [49]:
#retired
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='retired']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
1385,retired,169,"Wrong Trousers, The (1993)",5.0
1283,retired,408,"Close Shave, A (1995)",5.0
1302,retired,512,Wings of Desire (1987),5.0
844,retired,498,"African Queen, The (1951)",5.0
1543,retired,306,"Mrs. Brown (Her Majesty, Mrs. Brown) (1997)",5.0
...,...,...,...,...
1065,retired,527,Gandhi (1982),4.0
755,retired,488,Sunset Blvd. (1950),4.0
940,retired,770,Devil in a Blue Dress (1995),4.0
815,retired,505,Dial M for Murder (1954),4.0


In [50]:
#salesman
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='salesman']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
715,salesman,318,Schindler's List (1993),5.0
655,salesman,59,Three Colors: Red (1994),5.0
788,salesman,329,Desperate Measures (1998),5.0
648,salesman,209,This Is Spinal Tap (1984),5.0
647,salesman,191,Amadeus (1984),5.0
...,...,...,...,...
38,salesman,1016,Con Air (1997),4.5
593,salesman,134,Citizen Kane (1941),4.5
110,salesman,237,Jerry Maguire (1996),4.5
618,salesman,234,Jaws (1975),4.5


In [51]:
#scientist
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='scientist']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
1963,scientist,156,Reservoir Dogs (1992),5.000000
2017,scientist,650,"Seventh Seal, The (Sjunde inseglet, Det) (1957)",5.000000
1724,scientist,207,Cyrano de Bergerac (1990),5.000000
979,scientist,525,"Big Sleep, The (1946)",5.000000
1954,scientist,219,"Nightmare on Elm Street, A (1984)",5.000000
...,...,...,...,...
951,scientist,195,"Terminator, The (1984)",4.111111
88,scientist,22,Braveheart (1995),4.111111
270,scientist,515,"Boot, Das (1981)",4.111111
913,scientist,222,Star Trek: First Contact (1996),4.100000


In [52]:
#student
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='student']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
21652,student,1516,"Wedding Gift, The (1994)",5.000000
21909,student,968,"Inspector General, The (1949)",5.000000
21866,student,1514,Dream With the Fishes (1997),5.000000
5172,student,1375,"Cement Garden, The (1993)",5.000000
20540,student,320,Paradise Lost: The Child Murders at Robin Hood...,5.000000
...,...,...,...,...
17704,student,199,"Bridge on the River Kwai, The (1957)",4.272727
6557,student,174,Raiders of the Lost Ark (1981),4.267442
11890,student,531,Shine (1996),4.259259
8137,student,169,"Wrong Trousers, The (1993)",4.250000


In [53]:
#technician
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='technician']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
3315,technician,896,"Sweet Hereafter, The (1997)",5.000000
3466,technician,512,Wings of Desire (1987),5.000000
1083,technician,169,"Wrong Trousers, The (1993)",5.000000
3393,technician,611,Laura (1944),5.000000
136,technician,652,Rosencrantz and Guildenstern Are Dead (1990),5.000000
...,...,...,...,...
1217,technician,1119,Some Kind of Wonderful (1987),4.333333
2461,technician,124,Lone Star (1996),4.333333
2162,technician,12,"Usual Suspects, The (1995)",4.333333
2357,technician,223,Sling Blade (1996),4.333333


In [54]:
#writer
jobInfo=allUserInfo.loc[allUserInfo['occupation']=='writer']

jobInfoAndMovie=pd.merge(jobInfo, movies, on='MovieId')
jobInfoAndMovie['Rating Average']=jobInfoAndMovie.groupby(['Names'])['Rating'].transform('mean')

jobInfoAndMovie[['occupation','MovieId','Names','Rating Average']].sort_values(by='Rating Average', ascending=[False]).drop_duplicates('Names').head(100)

,occupation,MovieId,Names,Rating Average
4946,writer,652,Rosencrantz and Guildenstern Are Dead (1990),5.000000
5224,writer,130,Kansas City (1996),5.000000
5230,writer,963,Some Folks Call It a Sling Blade (1993),5.000000
4886,writer,18,"White Balloon, The (1995)",5.000000
4951,writer,1499,Grosse Fatigue (1994),5.000000
...,...,...,...,...
417,writer,511,Lawrence of Arabia (1962),4.222222
1467,writer,736,Shadowlands (1993),4.200000
4438,writer,19,Antonia's Line (1995),4.200000
5060,writer,615,"39 Steps, The (1935)",4.200000


In [55]:
# 영화의 장르를 확인해줍니다
name = ['MovieId','Names',"unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]
moviesAndGenre= pd.read_csv('ml-100k/u.item',sep = '\|',header = None,engine = 'python',usecols=[0,1,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19, 20,21,22,23],names = name)
moviesAndGenre

,MovieId,Names,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
#각 유져가 평가한 영화와 장르가 같이 출력이 되게 했습니다. 
userInfoAndMovieGenre=pd.merge(allUserInfo, moviesAndGenre, on='MovieId')
userInfoAndMovieGenre

,user-id,MovieId,Rating,TimeStamp,age,gender,occupation,zip-code,Names,unknown,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,242,3,881250949,49,M,writer,55105,Kolya (1996),0,...,0,0,0,0,0,0,0,0,0,0
1,305,242,5,886307828,23,M,programmer,94086,Kolya (1996),0,...,0,0,0,0,0,0,0,0,0,0
2,6,242,4,883268170,42,M,executive,98101,Kolya (1996),0,...,0,0,0,0,0,0,0,0,0,0
3,234,242,4,891033261,60,M,retired,94702,Kolya (1996),0,...,0,0,0,0,0,0,0,0,0,0
4,63,242,3,875747190,31,M,marketing,75240,Kolya (1996),0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,863,1679,3,889289491,17,M,student,60089,B. Monkey (1998),0,...,0,0,0,0,0,1,0,1,0,0
99996,863,1678,1,889289570,17,M,student,60089,Mat' i syn (1997),0,...,0,0,0,0,0,0,0,0,0,0
99997,863,1680,2,889289570,17,M,student,60089,Sliding Doors (1998),0,...,0,0,0,0,0,1,0,0,0,0
99998,896,1681,3,887160722,28,M,writer,91505,You So Crazy (1994),0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
#Count the number of ratings by each genre for 1) all users
#장르 값이 1로 저장이 되있으므로 각 장르에 대해 sum을 해주고, 내림차순으로 출력했습니다.

userInfoAndMovieGenre[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

Drama          39895
Comedy         29832
Action         25589
Thriller       21872
Romance        19461
Adventure      13753
Sci-Fi         12730
War             9398
Crime           8055
Children's      7182
Horror          5317
Mystery         5245
Musical         4954
Animation       3605
Western         1854
Film-Noir       1733
Fantasy         1352
Documentary      758
unknown           10
dtype: int64

In [58]:
#Count the number of ratings by each genre for 2) genders
#남성과 여성유져에 대해 gender를 뽑아내고 동일한 방법으로 수행합니다
#남자 유져
menInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['gender']=='M']
menInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

Drama          28887
Comedy         21764
Action         20147
Thriller       16786
Romance        13603
Adventure      10612
Sci-Fi         10101
War             7209
Crime           6261
Children's      4950
Horror          4120
Mystery         3931
Musical         3512
Animation       2610
Western         1483
Film-Noir       1348
Fantasy          989
Documentary      571
unknown            8
dtype: int64

In [59]:
#여성유져
womanInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['gender']=='F']
womanInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

Drama          11008
Comedy          8068
Romance         5858
Action          5442
Thriller        5086
Adventure       3141
Sci-Fi          2629
Children's      2232
War             2189
Crime           1794
Musical         1442
Mystery         1314
Horror          1197
Animation        995
Film-Noir        385
Western          371
Fantasy          363
Documentary      187
unknown            2
dtype: int64

In [60]:
# Count the number of ratings by each genre for 3) all occupations and sort them in descending order respectively

# (for문을 사용해 처리하고 싶었으나, 잘 해결하지 못했습니다)
occupation = pd.read_csv('ml-100k/u.occupation', sep="|", header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
print(occupation_list)
print(" ")

jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[0]]
print(occupation_list[0], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

['administrator', 'artist', 'doctor', 'educator', 'engineer', 'entertainment', 'executive', 'healthcare', 'homemaker', 'lawyer', 'librarian', 'marketing', 'none', 'other', 'programmer', 'retired', 'salesman', 'scientist', 'student', 'technician', 'writer']
 
administrator

Drama          3099
Comedy         2203
Action         1858
Thriller       1570
Romance        1539
Adventure      1020
Sci-Fi          942
War             786
Crime           565
Children's      462
Horror          391
Mystery         385
Musical         354
Animation       203
Western         168
Film-Noir       147
Fantasy          94
Documentary      48
unknown           0
dtype: int64

In [61]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[1]]
print(occupation_list[1], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

artist

Drama          957
Comedy         617
Action         528
Thriller       476
Romance        462
Adventure      311
Sci-Fi         308
War            225
Children's     202
Crime          201
Mystery        164
Musical        157
Animation      121
Horror         102
Film-Noir       64
Documentary     34
Western         29
Fantasy         24
unknown          0
dtype: int64

In [62]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[2]]
print(occupation_list[2], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

doctor

Drama          238
Comedy         168
Romance        133
Thriller       117
Action         110
Adventure       57
Sci-Fi          56
War             44
Crime           41
Children's      28
Mystery         27
Musical         27
Animation       22
Horror          18
Documentary     13
Film-Noir        9
Fantasy          7
Western          5
unknown          0
dtype: int64

In [63]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[3]]
print(occupation_list[3], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

educator

Drama          4281
Comedy         2708
Romance        2006
Action         1962
Thriller       1767
Adventure      1128
War            1005
Sci-Fi          986
Crime           710
Children's      593
Mystery         532
Musical         481
Horror          421
Animation       263
Film-Noir       202
Western         189
Fantasy         112
Documentary     107
unknown           1
dtype: int64

In [64]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[4]]
print(occupation_list[4], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

engineer

Drama          3153
Comedy         2438
Action         2277
Thriller       1712
Romance        1467
Adventure      1299
Sci-Fi         1221
War             858
Children's      629
Crime           611
Musical         441
Horror          414
Mystery         363
Animation       363
Western         180
Film-Noir       145
Fantasy         125
Documentary      51
unknown           1
dtype: int64

In [65]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[5]]
print(occupation_list[5], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

entertainment

Drama          804
Comedy         574
Thriller       554
Action         499
Romance        342
Sci-Fi         289
Adventure      249
Crime          207
Horror         192
War            174
Mystery        132
Children's     104
Musical         68
Animation       62
Film-Noir       47
Western         27
Fantasy         25
Documentary     23
unknown          1
dtype: int64

In [66]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[6]]
print(occupation_list[6], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

executive

Drama          1407
Comedy          951
Thriller        811
Action          808
Romance         592
Adventure       436
Sci-Fi          391
War             312
Crime           287
Children's      232
Mystery         203
Horror          155
Musical         131
Animation        91
Film-Noir        58
Fantasy          50
Western          49
Documentary      26
unknown           1
dtype: int64

In [67]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[7]]
print(occupation_list[7], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

healthcare

Drama          1297
Comedy          722
Action          577
Thriller        563
Romance         499
Adventure       323
Sci-Fi          266
War             230
Children's      213
Crime           200
Mystery         140
Musical         126
Horror          120
Animation        89
Film-Noir        50
Western          47
Documentary      45
Fantasy          38
unknown           0
dtype: int64

In [68]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[8]]
print(occupation_list[8], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

homemaker

Drama          104
Thriller        95
Comedy          93
Action          92
Romance         59
Adventure       41
Mystery         32
Sci-Fi          30
War             28
Children's      25
Crime           25
Musical         15
Horror           9
Animation        9
Film-Noir        4
Fantasy          3
Documentary      2
Western          0
unknown          0
dtype: int64

In [69]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[9]]
print(occupation_list[9], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

lawyer

Drama          543
Comedy         460
Action         283
Romance        272
Thriller       247
Adventure      142
War            142
Sci-Fi         141
Crime          107
Children's      83
Mystery         78
Musical         72
Horror          60
Film-Noir       36
Western         35
Animation       35
Fantasy         14
Documentary     10
unknown          0
dtype: int64

In [70]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[10]]
print(occupation_list[10], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

librarian

Drama          2557
Comedy         1576
Romance        1245
Action          988
Thriller        977
Adventure       536
Sci-Fi          508
War             481
Crime           370
Mystery         293
Children's      290
Musical         238
Horror          226
Animation       159
Film-Noir        96
Western          82
Fantasy          55
Documentary      48
unknown           0
dtype: int64

In [71]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[11]]
print(occupation_list[11], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

marketing

Drama          824
Comedy         535
Action         477
Thriller       442
Romance        383
Adventure      246
War            228
Sci-Fi         221
Children's     163
Crime          161
Mystery        128
Musical        114
Horror          86
Animation       79
Film-Noir       40
Western         35
Fantasy         20
Documentary     12
unknown          0
dtype: int64

In [72]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[12]]
print(occupation_list[12], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

none

Action         301
Drama          268
Thriller       262
Comedy         255
Adventure      160
Romance        150
Sci-Fi         145
Children's      91
Crime           74
Horror          56
War             55
Musical         43
Animation       43
Mystery         33
Fantasy         12
Film-Noir       12
Western         10
Documentary      1
unknown          1
dtype: int64

In [73]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[13]]
print(occupation_list[13], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

other

Drama          4307
Comedy         3165
Action         2672
Thriller       2321
Romance        2177
Adventure      1431
Sci-Fi         1242
War            1000
Children's      881
Crime           828
Mystery         579
Musical         566
Horror          535
Animation       421
Western         172
Film-Noir       146
Fantasy         144
Documentary      65
unknown           1
dtype: int64

In [74]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[14]]
print(occupation_list[14], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

programmer

Drama          2800
Comedy         2418
Action         2322
Thriller       1750
Romance        1400
Adventure      1240
Sci-Fi         1167
War             755
Crime           666
Children's      583
Horror          389
Musical         385
Mystery         375
Animation       312
Western         162
Film-Noir       141
Fantasy         117
Documentary      51
unknown           1
dtype: int64

In [75]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[15]]
print(occupation_list[15], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

retired

Drama          727
Comedy         512
Romance        321
Thriller       309
Action         278
Adventure      160
War            145
Crime          137
Sci-Fi         130
Children's     110
Mystery         98
Musical         93
Horror          66
Film-Noir       43
Animation       35
Western         33
Fantasy         22
Documentary     16
unknown          0
dtype: int64

In [76]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[16]]
print(occupation_list[16], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

salesman

Drama          317
Comedy         314
Action         222
Thriller       200
Romance        166
Adventure       99
Sci-Fi          98
Crime           79
War             77
Horror          49
Mystery         40
Children's      38
Musical         27
Western         16
Animation       15
Film-Noir       14
Fantasy          9
Documentary      2
unknown          0
dtype: int64

In [77]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[17]]
print(occupation_list[17], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

scientist

Drama          932
Action         566
Comedy         535
Thriller       469
Romance        440
Sci-Fi         318
Adventure      285
War            255
Crime          177
Mystery        122
Children's     108
Musical         91
Horror          70
Animation       61
Film-Noir       39
Western         24
Fantasy         20
Documentary     17
unknown          0
dtype: int64

In [78]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[18]]
print(occupation_list[18], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

student

Drama          7777
Comedy         6958
Action         6398
Thriller       5130
Romance        4156
Adventure      3358
Sci-Fi         3069
Crime          1836
War            1770
Children's     1759
Horror         1340
Musical        1078
Mystery        1009
Animation       914
Western         408
Fantasy         352
Film-Noir       269
Documentary     117
unknown           1
dtype: int64

In [79]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[19]]
print(occupation_list[19], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

technician

Drama          1295
Action         1079
Comedy         1056
Thriller        796
Romance         645
Adventure       558
Sci-Fi          531
War             383
Crime           275
Children's      221
Mystery         177
Musical         175
Horror          164
Animation       132
Western          89
Film-Noir        46
Fantasy          38
Documentary      24
unknown           1
dtype: int64

In [80]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[20]]
print(occupation_list[20], end="")
jobInfo[["unknown","Action","Adventure","Animation",
              "Children's","Comedy" ,"Crime" ,"Documentary","Drama","Fantasy","Film-Noir", "Horror" ,"Musical","Mystery" ,"Romance","Sci-Fi",
              "Thriller","War", "Western"]].sum().sort_values(ascending=[False])

writer

Drama          2208
Comedy         1574
Thriller       1304
Action         1292
Romance        1007
Adventure       674
Sci-Fi          671
Crime           498
Horror          454
War             445
Children's      367
Mystery         335
Musical         272
Animation       176
Film-Noir       125
Western          94
Fantasy          71
Documentary      46
unknown           1
dtype: int64

In [81]:
# Measure the average ratings by genre for 1) all users, 2) genders, and 3) all occupations and sort them in descending order respectively.

#우선 u.genre 파일을 읽어와 각 장르를 리스트 처리해줍니다.
genre = pd.read_csv('ml-100k/u.genre', sep="|", header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [82]:
#기존에 가지고 있던 모든 user의 정보를 이용해 각 장르별 평균을 출력하였습니다.
#그전에 한 방법으로 진행하면 매우 번거로울거 같아 새로운 데이터 프레임을 만들었습니다
#새로운 데이터 프레임을 만들어 각 장르를 1열, rating의 평균을 계산하여 2열에 넣어주었습니다

genre_averageForAllUser=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=userInfoAndMovieGenre[userInfoAndMovieGenre[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForAllUser.loc[len(genre_averageForAllUser)]=row
    
print("****Average rating for each genre rated by all users****")
genre_averageForAllUser.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by all users****


genre,average rating
Film-Noir,3.921523
War,3.815812
Drama,3.687379
Documentary,3.672823
Mystery,3.638132
Crime,3.632278
Romance,3.621705
Western,3.613269
Animation,3.576699
Sci-Fi,3.560723


In [83]:
#각 성별에 따른 평점 평균도 동일하게 수행하였습니다 (기존에 뽑아낸 남성, 여성유저 데이터 사용)

#남성유져
genre_averageForMen=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=menInfo[menInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForMen.loc[len(genre_averageForMen)]=row
    
print("****Average rating for each genre rated by Men****")
genre_averageForMen.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by Men****


genre,average rating
Film-Noir,3.973294
War,3.826328
Drama,3.696957
Documentary,3.691769
Mystery,3.664208
Crime,3.654049
Western,3.637896
Romance,3.607072
Sci-Fi,3.577072
Animation,3.557471


In [84]:
#여성 유져

genre_averageForWoman=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=womanInfo[womanInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForWoman.loc[len(genre_averageForWoman)]=row
    
print("****Average rating for each genre rated by Woman****")
genre_averageForWoman.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by Woman****


genre,average rating
War,3.781179
Film-Noir,3.740260
Drama,3.662246
Romance,3.655685
Musical,3.640083
Animation,3.627136
Documentary,3.614973
Mystery,3.560122
Crime,3.556299
Adventure,3.517988


In [85]:
#각 직업에 따른 평점 평균
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[0]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[0]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by administrator users****


genre,average rating
Film-Noir,3.972789
War,3.924936
Documentary,3.875000
Drama,3.796063
Mystery,3.763636
Romance,3.741391
Animation,3.738916
Western,3.702381
Crime,3.693805
Sci-Fi,3.650743


In [86]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[1]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[1]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by artist users****


genre,average rating
Film-Noir,4.234375
Documentary,4.147059
War,3.902222
Western,3.862069
Sci-Fi,3.818182
Mystery,3.792683
Drama,3.789969
Romance,3.729437
Crime,3.676617
Thriller,3.590336


In [87]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[2]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[2]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by doctor users****


genre,average rating
War,4.159091
Film-Noir,4.111111
Mystery,3.962963
Crime,3.804878
Western,3.800000
Drama,3.785714
Thriller,3.769231
Romance,3.699248
Action,3.618182
Musical,3.592593


In [88]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[3]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[3]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by educator users****


genre,average rating
Film-Noir,4.064356
War,3.908458
Western,3.851852
Drama,3.811259
Mystery,3.789474
Documentary,3.775701
Romance,3.758724
Crime,3.752113
Sci-Fi,3.622718
Adventure,3.619681


In [89]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[4]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[4]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by engineer users****


genre,average rating
Film-Noir,3.924138
War,3.839161
Drama,3.701554
Documentary,3.666667
Crime,3.666121
Romance,3.644853
Mystery,3.633609
Animation,3.614325
Western,3.600000
Sci-Fi,3.583948


In [90]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[5]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[5]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by entertainment users****


genre,average rating
unknown,4.000000
Film-Noir,3.914894
Documentary,3.869565
War,3.804598
Crime,3.710145
Drama,3.605721
Western,3.592593
Sci-Fi,3.529412
Romance,3.488304
Mystery,3.477273


In [91]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[6]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[6]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by executive users****


genre,average rating
War,3.823718
Western,3.673469
Sci-Fi,3.626598
Musical,3.625954
Film-Noir,3.603448
Mystery,3.600985
Animation,3.560440
Action,3.532178
Adventure,3.529817
Thriller,3.521578


In [92]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[7]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[7]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by healthcare users****


genre,average rating
Musical,3.277778
War,3.265217
Mystery,3.121429
Film-Noir,3.100000
Action,3.053726
Thriller,3.049734
Crime,3.045000
Sci-Fi,3.030075
Adventure,3.012384
Romance,2.995992


In [93]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[8]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[8]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by homemaker users****


genre,average rating
Mystery,3.687500
Drama,3.538462
Romance,3.525424
War,3.500000
Documentary,3.500000
Action,3.456522
Adventure,3.439024
Crime,3.360000
Thriller,3.357895
Film-Noir,3.250000


In [94]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[9]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[9]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by lawyer users****


genre,average rating
Film-Noir,4.500000
War,3.943662
Mystery,3.923077
Documentary,3.900000
Horror,3.866667
Musical,3.861111
Adventure,3.859155
Romance,3.834559
Thriller,3.813765
Drama,3.802947


In [95]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[10]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[10]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by librarian users****


genre,average rating
Film-Noir,3.968750
Documentary,3.812500
Drama,3.701603
Mystery,3.689420
War,3.683992
Crime,3.654054
Western,3.585366
Animation,3.553459
Romance,3.548594
Thriller,3.507677


In [96]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[11]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[11]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by marketing users****


genre,average rating
Film-Noir,4.100000
War,3.846491
Drama,3.608010
Mystery,3.585938
Documentary,3.583333
Sci-Fi,3.547511
Crime,3.515528
Western,3.514286
Action,3.484277
Romance,3.477807


In [97]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[12]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[12]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by none users****


genre,average rating
unknown,5.000000
War,4.072727
Documentary,4.000000
Animation,3.976744
Drama,3.932836
Romance,3.920000
Musical,3.860465
Horror,3.857143
Thriller,3.835878
Sci-Fi,3.806897


In [98]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[13]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[13]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by other users****


genre,average rating
unknown,4.000000
Film-Noir,3.842466
War,3.842000
Documentary,3.784615
Drama,3.719294
Animation,3.672209
Western,3.627907
Musical,3.627208
Romance,3.614607
Crime,3.595411


In [99]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[14]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[14]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by programmer users****


genre,average rating
unknown,4.000000
War,3.888742
Film-Noir,3.886525
Drama,3.755714
Western,3.746914
Documentary,3.725490
Sci-Fi,3.680377
Crime,3.669670
Animation,3.660256
Romance,3.643571


In [100]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[15]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[15]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by retired users****


genre,average rating
Film-Noir,4.023256
Mystery,3.714286
Animation,3.628571
War,3.627586
Crime,3.613139
Musical,3.526882
Western,3.515152
Drama,3.514443
Sci-Fi,3.507692
Romance,3.476636


In [101]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[16]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[16]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by salesman users****


genre,average rating
Documentary,4.000000
Drama,3.984227
War,3.948052
Western,3.937500
Crime,3.860759
Film-Noir,3.785714
Mystery,3.775000
Adventure,3.717172
Animation,3.666667
Action,3.653153


In [102]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[17]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[17]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by scientist users****


genre,average rating
Film-Noir,4.025641
Documentary,3.882353
War,3.815686
Drama,3.815451
Crime,3.813559
Mystery,3.729508
Animation,3.639344
Western,3.625000
Romance,3.615909
Sci-Fi,3.578616


In [103]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[18]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[18]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by student users****


genre,average rating
Film-Noir,3.832714
Documentary,3.829060
War,3.819209
Drama,3.702842
Crime,3.657407
Romance,3.650866
Animation,3.563457
Mystery,3.555005
Western,3.549020
Sci-Fi,3.531118


In [104]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[19]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[19]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by technician users****


genre,average rating
Film-Noir,4.021739
unknown,4.000000
Documentary,3.958333
War,3.715405
Drama,3.694981
Mystery,3.689266
Romance,3.631008
Sci-Fi,3.630885
Animation,3.606061
Crime,3.509091


In [105]:
jobInfo=userInfoAndMovieGenre.loc[userInfoAndMovieGenre['occupation']==occupation_list[20]]

genre_averageForeachJob=pd.DataFrame(columns=['genre', 'average rating'])
for gen in genre_list:
    genre_this=jobInfo[jobInfo[gen]==1]
    row=[gen, genre_this['Rating'].mean()]
    genre_averageForeachJob.loc[len(genre_averageForeachJob)]=row
    
print("****Average rating for each genre rated by {} users****".format(occupation_list[20]))
genre_averageForeachJob.sort_values('average rating', ascending=[False]).style.hide_index()

****Average rating for each genre rated by writer users****


genre,average rating
Film-Noir,3.968000
Documentary,3.782609
Mystery,3.629851
War,3.577528
Drama,3.511322
Crime,3.502008
Animation,3.500000
Musical,3.474265
Romance,3.469712
Western,3.414894
